# True Imagen Landsat

**DESCRIBIR DE QUE VA EL NOTEBOOK** las bandas que serán utilizadas, el orden necesario, las longitudes de onda etc.

**Importación de librerías**

In [19]:
import ee
import geemap
import os
import pandas as pd
import json
import requests
import rasterio
from matplotlib.colors import ListedColormap
from matplotlib_scalebar.scalebar import ScaleBar  # <-- for the scale bar
import matplotlib.pyplot as plt
import numpy as np

## 1) Autentificación


In [2]:
# 1) Obtención de la dirección de trabajo
direction = os.getcwd()

# Remove the last part of the path
direction = os.path.dirname(direction)

print("Dirección del proyecto:", direction)

# Dirección de la llave
service_account = direction + '/conf/local/gcp-for-data-science-397913-4fd843feede1.json'

# Autentificación
credentials = ee.ServiceAccountCredentials(email=None, key_file=service_account)
ee.Initialize(credentials)

Dirección del proyecto: /Users/orlandoandrade/Documents/Negocios/Satelites/selene


## 2) Importación de datos


**NOMBRES DE LAS IMAGENES BASEP**

In [3]:
images_names = pd.read_csv(filepath_or_buffer=direction + "/data/02_intermediate/sentinel_images_names.csv")
print("Total de imagenes:", len(images_names))
images_names.head(n=4)

Total de imagenes: 16


,Images names
0,Image_2024-10-04 00:00:00
1,Image_2024-10-14 00:00:00
2,Image_2024-10-19 00:00:00
3,Image_2024-10-24 00:00:00


**LECTURA DEL AREA DE INTERES (AOI)**

In [4]:
# Dirección
geojson_file =  direction + "/data/01_raw/AOI.geojson" # Creo que esta sección no es necesria aqui

# Step 1: Load the GeoJSON file
with open(geojson_file, "r") as f:
    geojson_data = json.load(f)

**Convertilor a una geometria de earth engine**


In [5]:
area_interest = ee.Geometry(geojson_data) # Creo que esta sección no es ya necesaria aqui
type(area_interest)

ee.geometry.Geometry

## 3) Procesamiento


**CREACION DEL MAPA WEB BASE**

Se utilizara para inspeccionar las imagenes

### 3.1) Generación de imagenes reales

In [23]:
# diccionario de imagenes, se usara para el almacenamiento de los resultados
images_dic = {}

# Creación del mapa web base
# Create a map using geemap
Map = geemap.Map(zoom=10) # Tengo que ponerle algún centroide para iniciar más cercano en el área de interés

# 1) Inicio del ciclo
for i in images_names["Images names"]:

    # 1.1) Path to the image in the GCS bucket
    gcs_path = f'gs://earth_engine_selene/sentinel2/Row_Data/{i}.tif' # Crear más bien un GCP path base

    # 1.2) Load the image from the GCS bucket
    image = ee.Image.loadGeoTIFF(gcs_path)

    # 1.3) Selección de las bandas necesrias
    true_color = image.select(selectors=["B4", "B3", "B2"])

    # 1.4) Define visualization parameters
    vis_params = {
        "min": 0,
        "max": 3000,
        "bands": ["B4", "B3", "B2"]}

    # 1.5) Integración de la imagen a un mapa interactivo
    Map.addLayer(ee_object=true_color, vis_params= vis_params,
               name=f"Sentinel-2 True Color {i}",
               shown=False)

    # 1.6) Obtección del URL de la  imagen real (la descargaremos como PNG)
    url = true_color.getThumbURL(vis_params)

    # 1.7) Exportación de la imagen a PNG
    # Download the image and save it locally
    output_file = direction + f"/data/07_model_output/Sentinel/RealImage/True_color {i}.png"
    response = requests.get(url=url)
    if response.status_code == 200:
        with open(file=output_file, mode='wb') as f: #string "output_file" para exportar en esa direccion la imagen
            f.write(response.content) # Esto es lo que exportara que es el contenido de la respuesta, la imagen
        print(f"Image successfully downloaded as '{output_file}'")
    else:
        print(f"Error: Unable to download the image. HTTP status code {response.status_code}")

    # 1.8) Exportacióm de las imagenes reales a GCP
    # Indicamos la tarea de exportacion
    task = ee.batch.Export.image.toCloudStorage(
    image=true_color,
    #description=f"Export", # Este es el nombre del archivo que se exportara , si vas a usar el filenameprexi no es necesario esto
    bucket='earth_engine_selene',  # Cambia esto al nombre de tu bucket
    fileNamePrefix=f"sentinel2/True_Color/True_Color {i}", # Damos una dirección dentro del bucket y le damos nombre
    scale=10,  # Resolución en metros
    fileFormat='GeoTIFF')  # Formato de archivo

    # Inicio de la exportación
    task.start()  # Inicia la tarea de exportación
    print("Se exportó correctamente la imagen a Cloud Storage:", i)

# Exportación del HTML
html_file = direction + "/data/07_model_output/Sentinel/RealImage/True_color_Map.html"
Map.to_html(filename=html_file, title="True color image", width="100%", height="880px") # Exporacion a una html

Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/Sentinel/RealImage/True_color Image_2024-10-04 00:00:00.png'
Se exportó correctamente la imagen a Cloud Storage: Image_2024-10-04 00:00:00
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/Sentinel/RealImage/True_color Image_2024-10-14 00:00:00.png'
Se exportó correctamente la imagen a Cloud Storage: Image_2024-10-14 00:00:00
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/Sentinel/RealImage/True_color Image_2024-10-19 00:00:00.png'
Se exportó correctamente la imagen a Cloud Storage: Image_2024-10-19 00:00:00
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/Sentinel/RealImage/True_color Image_2024-10-24 00:00:00.png'
Se exportó correctamente la imagen a Cloud Storage: Image_2024-10-24 0

### 3.2) Porcesamiento de imagenes reales
En este sección se integrarán aspectos adicionales a las imagenes como lo son título y escala

In [28]:
# Autentificacion , variables del sistema
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =  service_account

In [33]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar  # For the scale bar

# Loop through image names
for i in images_names["Images names"]:

    true_color_path = f"gs://earth_engine_selene/sentinel2/True_Color/True_Color {i}.tif"

    # Process True Color Image
    with rasterio.open(true_color_path) as src:
        raster_data = src.read([1, 2, 3])  # Assuming RGB bands
        pixel_size_x = abs(src.transform[0])

        fig, ax = plt.subplots(figsize=(8, 6))
        img = show(raster_data, ax=ax)

        scalebar = ScaleBar(pixel_size_x, units='m', location='lower left', length_fraction=0.25)
        ax.add_artist(scalebar)

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_title(f"True Color Image {i}")

        fig.savefig(direction + f"/data/07_model_output/Sentinel/RealImage/TrueColor_Procesado{i}.png")
        plt.close(fig)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0..5760].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0..9824].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0..5936].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0..5376].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0..5272].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0..5472].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0..5356].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats 

In [36]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

# Loop through image names
for i in images_names["Images names"]:
    true_color_path = f"gs://earth_engine_selene/sentinel2/True_Color/True_Color {i}.tif"

    with rasterio.open(true_color_path) as src:
        raster_data = src.read([3, 2, 1]).astype(np.float32)  # Convert to float for normalization

        # Normalize values if necessary
        if raster_data.max() > 255:
            raster_data = (raster_data / raster_data.max()) * 255
            raster_data = raster_data.astype(np.uint8)

        # Swap from BGR to RGB (if needed)
        raster_data = raster_data[[2, 1, 0], :, :]

        # Handle NoData values
        no_data_value = src.nodata
        if no_data_value is not None:
            raster_data = np.where(raster_data == no_data_value, 0, raster_data)

        # Get pixel size for scale bar
        pixel_size_x = abs(src.transform[0])

        fig, ax = plt.subplots(figsize=(8, 6))
        img = show(raster_data, ax=ax)

        # Add scale bar
        scalebar = ScaleBar(pixel_size_x, units='m', location='lower left', length_fraction=0.25)
        ax.add_artist(scalebar)

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_title(f"True Color Image {i}")

        fig.savefig(direction + f"/data/07_model_output/Sentinel/RealImage/TrueColor_Procesado_{i}.png")
        plt.close(fig)


In [ ]:
## NOTA:  Falta modficar los outputs a la carpeta de RealImage, hacer 2 subcarpperas una paras la imagenes solas y otra procesadas (ya con titulo. y escala)
a = "nota"